In [1]:
from pia_bench.bench import PiaBenchMark

benchmark_path = "assets/huggingface_benchmarks_dataset"
pia_benchmark = PiaBenchMark(benchmark_path , "assets/huggingface_benchmarks_dataset/CFG/topk.json" )
pia_benchmark.preprocess_structure()

print("Categories identified:", pia_benchmark.categories)

/home/jungseoik/miniconda3/envs/vlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Folder preprocessing completed.
Categories identified: []


In [ ]:
pia_benchmark.extract_visual_vector()

In [ ]:
import os
import numpy as np
import torch
from typing import Dict, List, Tuple
from devmacs_core.devmacs_core import DevMACSCore
from devmacs_core.utils.common.cal import scale_sim, loose_similarity
from utils.parser import load_config, PromptManager

class EventDetector:
    def __init__(self, config_path: str):
        self.config = load_config(config_path)
        self.macs = DevMACSCore()
        self.prompt_manager = PromptManager(config_path)
        self.sentences = self.prompt_manager.sentences
        self.text_vectors = self.macs.get_text_vector(self.sentences)
        self.result_pred = None
    
    def process_video_vectors(self, base_dir: str) -> Dict:
        results = {}
        
        for category in os.listdir(base_dir):
            category_path = os.path.join(base_dir, category)
            if not os.path.isdir(category_path):
                continue
                
            results[category] = {}
            for file in os.listdir(category_path):
                if file.endswith('.npy'):
                    video_name = os.path.splitext(file)[0]
                    file_path = os.path.join(category_path, file)
                    results[category][video_name] = self._process_single_vector(file_path)
        
        self.result_pred = results
        return results
    
    def _process_single_vector(self, vector_path: str) -> Dict:
        video_vector = np.load(vector_path)
        processed_vectors = []
        frame_interval = 15  

        for vector in video_vector:
            v = vector.squeeze(0)  # (1, 512) -> (512,)
            v = torch.from_numpy(v).unsqueeze(0).cuda()  # (512,) -> (1, 512) # GPU로 이동
            processed_vectors.append(v)
            
        frame_results = {}
        for vector_idx, v in enumerate(processed_vectors):
            actual_frame = vector_idx * frame_interval  # 실제 프레임 번호 계산
            sim_scores = loose_similarity(
                sequence_output=self.text_vectors.cuda(),  # text vectors도 GPU로
                visual_output=v.unsqueeze(1)  # (1, 512) -> (1, 1, 512)
            )
            frame_results[actual_frame] = self._calculate_alarms(sim_scores)
        return frame_results

    
    def _calculate_alarms(self, sim_scores: torch.Tensor) -> Dict:
        """
        유사도 점수를 기반으로 각 이벤트의 알람 상태 계산
        Returns:
            Dict: {
                '이벤트명': {
                    'alarm': 0 또는 1,  # 0: Normal, 1: Alarm
                    'scores': [스코어들],
                    'top_k_types': [타입들]
                }
            }
        """
        event_alarms = {}
        
        for event_config in self.config['PROMPT_CFG']:
            event = event_config['event']
            top_k = event_config['top_candidates']
            threshold = event_config['alert_threshold']
            
            event_prompts = self._get_event_prompts(event)
            event_scores = sim_scores[event_prompts['indices']]
            
            top_k_values, top_k_indices = torch.topk(event_scores, min(top_k, len(event_scores)))
            
            abnormal_count = sum(1 for idx in top_k_indices 
                            if event_prompts['types'][idx.item()] == 'abnormal')
            
            event_alarms[event] = {
                'alarm': 1 if abnormal_count >= threshold else 0,  # 0: Normal, 1: Alarm
                'scores': top_k_values.tolist(),
                'top_k_types': [event_prompts['types'][idx.item()] for idx in top_k_indices]
            }
            
        return event_alarms
    
    def _get_event_prompts(self, event: str) -> Dict:
        """
        특정 이벤트의 모든 프롬프트 정보 반환
        """
        indices = []
        types = []
        
        for event_idx, event_config in enumerate(self.config['PROMPT_CFG']):
            if event_config['event'] == event:
                for status in ['normal', 'abnormal']:
                    for prompt_idx in range(len(event_config['prompts'][status])):
                        indices.append(len(indices))  # 실제 인덱스로 변환 필요
                        types.append(status)
                        
        return {'indices': indices, 'types': types}

detector = EventDetector('assets/huggingface_benchmarks_dataset/CFG/topk.json')
results = detector.process_video_vectors(pia_benchmark.vector_video_path)

for category, videos in results.items():
    print(f"\nCategory: {category}")
    for video_name, frames in videos.items():
        print(f"\nVideo: {video_name}")
        for frame_idx, alarms in frames.items():
            print(f"\nFrame {frame_idx}:")
            for event, status in alarms.items():
                print(f"{event}: {status['alarm']}")

In [3]:
import os
import numpy as np
import torch
from typing import Dict, List, Tuple
from devmacs_core.devmacs_core import DevMACSCore
from devmacs_core.utils.common.cal import scale_sim, loose_similarity
from utils.parser import load_config, PromptManager
import json
import pandas as pd
from tqdm import tqdm
import logging
from datetime import datetime

class EventDetector:
    def __init__(self, config_path: str):
        self.config = load_config(config_path)
        self.macs = DevMACSCore()
        self.prompt_manager = PromptManager(config_path)
        self.sentences = self.prompt_manager.sentences
        self.text_vectors = self.macs.get_text_vector(self.sentences)
        
    def process_and_save_predictions(self, vector_base_dir: str, label_base_dir: str, save_base_dir: str):
        """비디오 벡터를 처리하고 결과를 CSV로 저장"""
        

            # 전체 비디오 파일 수 계산
        total_videos = sum(len([f for f in os.listdir(os.path.join(vector_base_dir, d)) 
                                if f.endswith('.npy')]) 
                            for d in os.listdir(vector_base_dir) 
                            if os.path.isdir(os.path.join(vector_base_dir, d)))
        pbar = tqdm(total=total_videos, desc="Processing videos")
        
        for category in os.listdir(vector_base_dir):
            category_path = os.path.join(vector_base_dir, category)
            if not os.path.isdir(category_path):
                continue
            
            # 저장 디렉토리 생성
            save_category_dir = os.path.join(save_base_dir, category)
            os.makedirs(save_category_dir, exist_ok=True)
            
            for file in os.listdir(category_path):
                if file.endswith('.npy'):
                    video_name = os.path.splitext(file)[0]
                    vector_path = os.path.join(category_path, file)
                    
                    # 라벨 파일 읽기
                    label_path = os.path.join(label_base_dir, category, f"{video_name}.json")
                    with open(label_path, 'r') as f:
                        label_data = json.load(f)
                        total_frames = label_data['video_info']['total_frame']
                    
                    # 예측 결과 생성 및 저장
                    self._process_and_save_single_video(
                        vector_path=vector_path,
                        total_frames=total_frames,
                        save_path=os.path.join(save_category_dir, f"{video_name}.csv")
                    )
                    pbar.update(1)
        pbar.close()

    def _process_and_save_single_video(self, vector_path: str, total_frames: int, save_path: str):
        """단일 비디오 처리 및 저장"""
        # 기본 예측 수행
        sparse_predictions = self._process_single_vector(vector_path)
        
        # 데이터프레임으로 변환 및 확장
        df = self._expand_predictions(sparse_predictions, total_frames)
        
        # CSV로 저장
        df.to_csv(save_path, index=False)

    def _process_single_vector(self, vector_path: str) -> Dict:
        """기존 예측 로직"""
        video_vector = np.load(vector_path)
        processed_vectors = []
        frame_interval = 15
        
        for vector in video_vector:
            v = vector.squeeze(0)  # numpy array
            v = torch.from_numpy(v).unsqueeze(0).cuda()  # torch tensor로 변환 후 GPU로
            processed_vectors.append(v)
        
        frame_results = {}
        for vector_idx, v in enumerate(processed_vectors):
            actual_frame = vector_idx * frame_interval
            sim_scores = loose_similarity(
                sequence_output=self.text_vectors.cuda(),
                visual_output=v.unsqueeze(1)
            )
            frame_results[actual_frame] = self._calculate_alarms(sim_scores)
            
        return frame_results

    def _expand_predictions(self, sparse_predictions: Dict, total_frames: int) -> pd.DataFrame:
        """예측을 전체 프레임으로 확장"""
        # 카테고리 목록 추출 (첫 번째 프레임의 알람 결과에서)
        first_frame = list(sparse_predictions.keys())[0]
        categories = list(sparse_predictions[first_frame].keys())
        
        # 전체 프레임 생성
        df = pd.DataFrame({'frame': range(total_frames)})
        
        # 각 카테고리에 대한 알람 값 초기화
        for category in categories:
            df[category] = 0
        
        # 예측값 채우기
        frame_keys = sorted(sparse_predictions.keys())
        for i in range(len(frame_keys)):
            current_frame = frame_keys[i]
            next_frame = frame_keys[i + 1] if i + 1 < len(frame_keys) else total_frames
            
            # 각 카테고리의 알람 값 설정
            for category in categories:
                alarm_value = sparse_predictions[current_frame][category]['alarm']
                df.loc[current_frame:next_frame-1, category] = alarm_value
        
        return df


    def _calculate_alarms(self, sim_scores: torch.Tensor) -> Dict:
        """유사도 점수를 기반으로 각 이벤트의 알람 상태 계산"""
        # 로거 설정
        log_filename = f"alarm_calculation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        logger = logging.getLogger(__name__)
        
        event_alarms = {}
        
        for event_config in self.config['PROMPT_CFG']:
            event = event_config['event']
            top_k = event_config['top_candidates']
            threshold = event_config['alert_threshold']
            
            logger.info(f"\nProcessing event: {event}")
            logger.info(f"Top K: {top_k}, Threshold: {threshold}")
            
            event_prompts = self._get_event_prompts(event)

            # logger.debug(f"\nEvent Prompts Debug for {event}:")
            # logger.debug(f"Indices: {event_prompts['indices']}")
            # logger.debug(f"Types: {event_prompts['types']}")
            # logger.debug(f"\nSim Scores Debug:")
            # logger.debug(f"Shape: {sim_scores.shape}")
            # logger.debug(f"Raw scores: {sim_scores}")
            event_scores = sim_scores[event_prompts['indices']]
            # logger.debug(f"Event scores shape: {event_scores.shape}")
            # logger.debug(f"Event scores: {event_scores}")
            # 각 프롬프트와 점수 출력
            logger.info("\nAll prompts and scores:")
            for idx, (score, prompt_type) in enumerate(zip(event_scores, event_prompts['types'])):
                logger.info(f"Type: {prompt_type}, Score: {score.item():.4f}")
            
            top_k_values, top_k_indices = torch.topk(event_scores, min(top_k, len(event_scores)))
            
            # Top K 결과 출력
            logger.info(f"\nTop {top_k} selections:")
            for idx, (value, index) in enumerate(zip(top_k_values, top_k_indices)):
                # indices[index]가 아닌 index를 직접 사용
                prompt_type = event_prompts['types'][index]  # 수정된 부분
                logger.info(f"Rank {idx+1}: Type: {prompt_type}, Score: {value.item():.4f}")

            abnormal_count = sum(1 for idx in top_k_indices 
                    if event_prompts['types'][idx] == 'abnormal')  # 수정된 부분
            # for idx, (value, orig_idx) in enumerate(zip(top_k_values, top_k_indices)):
            #     prompt_type = event_prompts['types'][orig_idx.item()]
            #     logger.info(f"Rank {idx+1}: Type: {prompt_type}, Score: {value.item():.4f}")
            
            # abnormal_count = sum(1 for idx in top_k_indices 
            #                 if event_prompts['types'][idx.item()] == 'abnormal')
            
            # 알람 결정 과정 출력
            logger.info(f"\nAbnormal count: {abnormal_count}")
            alarm_result = 1 if abnormal_count >= threshold else 0
            logger.info(f"Final alarm decision: {alarm_result}")
            logger.info("-" * 50)
            
            event_alarms[event] = {
                'alarm': alarm_result,
                'scores': top_k_values.tolist(),
                'top_k_types': [event_prompts['types'][idx.item()] for idx in top_k_indices]
            }
        
        # 로거 종료
        logging.shutdown()
                
        return event_alarms
    
    def _get_event_prompts(self, event: str) -> Dict:
        """
        특정 이벤트의 모든 프롬프트 정보 반환
        """
        indices = []
        types = []
        
        for event_idx, event_config in enumerate(self.config['PROMPT_CFG']):
            if event_config['event'] == event:
                for status in ['normal', 'abnormal']:
                    for prompt_idx in range(len(event_config['prompts'][status])):
                        indices.append(len(indices))  # 실제 인덱스로 변환 필요
                        types.append(status)
                        
        return {'indices': indices, 'types': types}
    
detector = EventDetector('assets/huggingface_benchmarks_dataset/CFG/topk.json')
detector.process_and_save_predictions(
    vector_base_dir='assets/huggingface_benchmarks_dataset/vector/video',
    label_base_dir='assets/huggingface_benchmarks_dataset/dataset',
    save_base_dir='assets/huggingface_benchmarks_dataset/alram'
)


/home/jungseoik/data/Abnormal_situation_leader_board/DevMACS-AI-solution-devmacs/Package-Common-AI-pia_ai_package/packages/pia/ai/tasks/T2VRet/models/clip4clip/main.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any is

In [5]:
import json
from typing import Dict, List

class AlarmExpander:
    def __init__(self, category_results: Dict, video_metadata_path: str):
        """
        category_results: Detector의 결과로 나온 알람 값
        video_metadata_path: 라벨링 파일이 저장된 경로
        """
        self.category_results = category_results
        self.video_metadata_path = video_metadata_path

    def expand_alarms(self) -> Dict:
        """
        비디오의 알람 값을 전체 프레임에 대해 확장.

        """
        expanded_results = {}

        for category, videos in self.category_results.items():
            expanded_results[category] = {}
            for video_name, alarms in videos.items():
                expanded_alarms = {}
                
                # 라벨링 파일에서 비디오 총 프레임 수 읽기
                label_path = f"{self.video_metadata_path}/{category}/{video_name}.json"
                with open(label_path, "r", encoding="utf-8") as f:
                    video_metadata = json.load(f)
                    total_frames = video_metadata["video_info"]["total_frame"]
                
                # 프레임 간격 확장
                frame_keys = sorted(alarms.keys())
                for i in range(len(frame_keys)):
                    current_frame = frame_keys[i]
                    if i + 1 < len(frame_keys):
                        next_frame = frame_keys[i + 1]
                    else:
                        next_frame = total_frames

                    for frame in range(current_frame, next_frame):
                        expanded_alarms[frame] = alarms[current_frame]
                
                expanded_results[category][video_name] = expanded_alarms
        
        return expanded_results

In [ ]:
# Detector 결과와 라벨링 파일 경로 설정
detector_results = {
    "F": {
        "cat copy 2": {
            0: {"D": 0, "S": 0, "V": 0, "F": 0},
            15: {"D": 0, "S": 0, "V": 0, "F": 0},
            30: {"D": 0, "S": 0, "V": 0, "F": 0},
        }
    }
}

label_path = "assets/huggingface_benchmarks_dataset/dataset"
# expander = AlarmExpander(detector_results, label_path)
expander = AlarmExpander(results, label_path)

expanded_results = expander.expand_alarms()
print(expanded_results)


In [ ]:
import os
import json
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from typing import Dict, List, Tuple

class MetricsEvaluator:
    def __init__(self, expanded_results: Dict, label_dir: str):
        self.expanded_results = expanded_results
        self.label_dir = label_dir
        self.metrics = {}

    def load_label_file(self, category: str, video_name: str) -> Dict:
        """라벨링 파일 로드"""
        label_path = os.path.join(self.label_dir, category, f"{video_name}.json")
        with open(label_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    def create_frame_level_ground_truth(self, label_data: Dict, category: str, total_frames: int) -> np.ndarray:
        """타임스탬프(프레임) 기반으로 ground truth 생성"""
        ground_truth = np.zeros(total_frames)
        
        for clip in label_data['clips']:
            clip_data = list(clip.values())[0]
            if clip_data['category'] == category:
                start_frame = clip_data['timestamp'][0]  # 직접 프레임 인덱스
                end_frame = clip_data['timestamp'][1]    # 직접 프레임 인덱스
                ground_truth[start_frame:end_frame+1] = 1
        
        return ground_truth

    def calculate_metrics(self, y_true: np.ndarray, y_pred: np.ndarray) -> Dict:
        """성능 지표 계산"""
        # 특이도 계산을 위한 TN, FP
        tn = np.sum((y_true == 0) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        
        metrics = {
            'f1': f1_score(y_true, y_pred, zero_division=0),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'accuracy': accuracy_score(y_true, y_pred),
            'specificity': tn / (tn + fp) if (tn + fp) > 0 else 0
        }
        
        return metrics
    def evaluate(self) -> Dict:
        """전체 평가 수행"""
        for category, videos in self.expanded_results.items():
            self.metrics[category] = {}
            
            for video_name, frame_results in videos.items():
                try:
                    # 라벨 파일 로드
                    label_data = self.load_label_file(category, video_name)
                    total_frames = label_data['video_info']['total_frame']
                    
                    # Ground truth 생성
                    ground_truth = self.create_frame_level_ground_truth(label_data, category, total_frames)
                    
                    # 예측값 배열 생성 - 수정된 부분
                    predictions = np.zeros(total_frames)
                    for frame_idx, alarms in frame_results.items():
                        # alarms[category]['alarm']에서 알람 값 가져오기
                        predictions[int(frame_idx)] = alarms[category]['alarm']
                    
                    # 디버깅을 위한 출력
                    print(f"\nProcessing {video_name} in category {category}")
                    print(f"Total frames: {total_frames}")
                    print(f"Ground truth sum: {np.sum(ground_truth)}")
                    print(f"Predictions sum: {np.sum(predictions)}")
                    # print(f"Ground truth samples: {ground_truth[:10]}")
                    # print(f"Predictions samples: {predictions[:10]}")
                    print(f"Ground truth values: {np.where(ground_truth == 1)[0]}")  # 1인 프레임의 인덱스
                    print(f"Prediction values: {np.where(predictions == 1)[0]}")     # 1인 프레임의 인덱스
                    print(f"All prediction values: {predictions}")  # 전체 예측값
                    # 메트릭 계산
                    video_metrics = self.calculate_metrics(ground_truth, predictions)
                    self.metrics[category][video_name] = video_metrics
                    
                    print(f"Metrics for {video_name}: {video_metrics}")
                    
                except Exception as e:
                    print(f"Error processing {video_name} in category {category}: {str(e)}")
                    continue
            
        return self.metrics

# 사용 예시:
def evaluate_results(expanded_results: Dict, label_dir: str) -> Dict:
    evaluator = MetricsEvaluator(expanded_results, label_dir)
    return evaluator.evaluate()

# 평균 성능 계산을 위한 함수
def calculate_average_metrics(metrics: Dict) -> Dict:
    """각 카테고리별 평균 성능 계산"""
    avg_metrics = {}
    
    for category, video_metrics in metrics.items():
        avg_metrics[category] = {
            metric: np.mean([vm[metric] for vm in video_metrics.values() if metric in vm])
            for metric in ['f1', 'precision', 'recall', 'accuracy', 'specificity']
        }
    
    return avg_metrics

# 실행
label_dir = "assets/PIA_FSV copy/dataset"
metrics = evaluate_results(expanded_results, label_dir)
avg_metrics = calculate_average_metrics(metrics)

# 결과 출력
for category, category_metrics in metrics.items():
    print(f"\nCategory: {category}")
    for video_name, video_metrics in category_metrics.items():
        print(f"\n{video_name}:")
        for metric_name, value in video_metrics.items():
            print(f"{metric_name}: {value:.3f}")

print("\nAverage Metrics:")
for category, category_avg in avg_metrics.items():
    print(f"\n{category}:")
    for metric_name, value in category_avg.items():
        print(f"{metric_name}: {value:.3f}")